# Aardvark Metadata for HDX

This script aims to harvest metadata in Aardvark version from datasets on [HDX site](https://data.humdata.org).

> Originally created by Gene Cheng [@Ziiiiing]() on Oct 24, 2021

In [1]:
import csv 
import time
import uuid
import geocoder
import urllib.request
from bs4 import BeautifulSoup

Before execute the script, you need to manually change the following cell to your own **GeoNames** user account. Or you can register a free one [here](http://www.geonames.org/login)

*Note: according to Terms and Conditions, the hourly limit for personal account is 1000 credits and 1 credit is 1 hit for webservice request.*

In [2]:
# Manual changes here
geonames_acc = 'geobtaa'

### STEP 1: Find All Data Links from Search Page

It seems like there are less than 200 data published on this site, so we set the `ext_page_size=200` to the `home_url` to get all search results in one page.

Next, we use the **Beautiful Soup** to find and store all data links in a list.

In [3]:
home_url = "https://data.humdata.org/dataset?ext_geodata=1&groups=usa&q=&sort=if(gt(last_modified%2Creview_date)%2Clast_modified%2Creview_date)%20desc&ext_page_size=600"
home_page = urllib.request.urlopen(home_url).read()
soup = BeautifulSoup(home_page, "html.parser")

# find geodata links
data_urls = []
linkFields = soup.find_all('div', {'class': 'dataset-heading'})
for tag in linkFields:
    url = 'https://data.humdata.org' + tag.find('a')['href']
    data_urls.append(url)

### STEP 2: Extract metadata from Each Data Page

In [4]:
def find_download(files):
    datasets = []
    for file in files:
        download = 'https://data.humdata.org' + file.find('a', href=True)['href']
        ftype = ''
        fsize = ''
        spans = file.find('a', {'class': 'heading'}).find_all('span')
        for span in spans:
            if span['class'] == ['format-label']:
                ftype = span['data-format']
            if span['class'] == ['format-filesize-label']:
                fsize = span.text.strip()[1:-1]
        datasets.append([download, ftype, fsize])
    
    # consider the first shapefile as download file
    for data in datasets:
        if data[1] == 'shp':
            ftype_full = 'Shapefile'
            return data[0], ftype_full, data[2]
    
    # if no shapefile exists, try to find ARC/INFO Grid, GeoTIFF, Geodatabase, Geopackage files instead
    for data in datasets:
        if data[1] in ['arc/info grid', 'geotiff', 'geodatabase', 'geopackage']:
            ftype_full = data[1].capitalize()
            return data[0], ftype_full, data[2]
        
    # else, return nothing
    return '','',''
        

In [5]:
# get the bounding box for the given location
# def find_bbox(location):
#     if location == 'World':
#         bbox = '-180,90,180,-90'
#         return bbox
#     # for single place, return bbox directly
#     if len(location.split('|')) == 1:
#         g1 = geocoder.geonames(location, key=geonames_acc)
#         gid = g1.geonames_id
#         g2 = geocoder.geonames(gid, method='details', key=geonames_acc)
#         bbox = g2.bbox
#         w = str(round(bbox['southwest'][1],4))
#         n = str(round(bbox['northeast'][0],4))
#         e = str(round(bbox['northeast'][1],4))
#         s = str(round(bbox['southwest'][0],4))
#         return ','.join((w,n,e,s))
#     # for multiple locations, find a broader bounding box for all places
#     else:
#         places = location.split('|')
#         w_all = 180
#         n_all = -90
#         e_all = -180
#         s_all = 90
#         for place in places:
#             try:
#                 g1 = geocoder.geonames(place, key=geonames_acc)
#                 gid = g1.geonames_id
#                 g2 = geocoder.geonames(gid, method='details', key=geonames_acc)
#                 bbox = g2.bbox
                
#                 w = round(bbox['southwest'][1],4)
#                 n = round(bbox['northeast'][0],4)
#                 e = round(bbox['northeast'][1],4)
#                 s = round(bbox['southwest'][0],4)

#                 if w < w_all:
#                     w_all = w
#                 if n > n_all:
#                     n_all = n
#                 if e > e_all:
#                     e_all = e
#                 if s < s_all:
#                     s_all = s
            
#             except:
#                 continue          

#         return ','.join((str(w_all),str(n_all),str(e_all),str(s_all)))


In [6]:
def collect_metadata(url):
    data_page = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(data_page, "html.parser")
    metadata = []
    
    alternativeTitle = soup.find('h1', {'class': 'itemTitle dataset-title'}).text.strip()
    title = alternativeTitle
    
    descriptionField = soup.find('div', {'class': 'notes embedded-content'}).find_all('p')
    description = ''.join(x.text.strip() for x in descriptionField)

    creator = soup.find('th', text = 'Contributor').findNext('td').text.strip()

    keywordField = soup.find('th', text = 'Tags').findNext('td').find_all('a')
    keyword = keyword = '|'.join(x.text.strip() for x in keywordField)
    
    try:
        updatedField = soup.find('th', text='Updated').findNext('td').text.strip()
        dd = updatedField.split()[0].zfill(2)
        yyyy = updatedField.split()[2]
        mm = str(time.strptime(updatedField.split()[1], '%B').tm_mon).zfill(2)
        dateIssued = '-'.join((yyyy,mm,dd))
    except:
        dateIssued = ''

    try:
        temporalCoverage = soup.find('th', text='Date of Dataset').findNext('td').text.strip()
        fromY = temporalCoverage.split('-')[0].split()[-1]
        toY = temporalCoverage.split('-')[1].split()[-1]
        dateRange = '-'.join((fromY, toY)) 
    except:
        temporalCoverage = ''
        dateRange = ''
    
    updateFrequency = soup.find('th', text='Expected Update Frequency').findNext('td').text.strip()

    locationField = soup.find('th', text='Location').findNext('td').find_all('a')
    spatialCoverage = '|'.join(x.text.strip() for x in locationField)
#     bbox = find_bbox(spatialCoverage) 
    
    license = soup.find('th', text='License').findNext('td').text.replace('\n', '').replace('\t', '').strip()
    

    files = soup.find_all('li', {'class': 'resource-item'})
    download, formatElement, fileSize = find_download(files)
    
    resourceType = 'Vector data'
    resourceClass = 'Datasets'
    information = url
    identifier = url
    idElement = str(uuid.uuid4())
    isoTopCat = ''
    language = 'eng'
    provider = 'University of Minnesota'
    code = '99-1400'
    memberOf = '99-1400'
    status = 'Active'
    accrualMethod = 'HTML'
    dateAccessioned = time.strftime("%Y-%m-%d")
    rights = ''
    accessRights = 'Public'
    suppressed = 'FALSE'
    childRecord = 'FALSE'
    
    metadata = [title, alternativeTitle, description, language, creator, 
                resourceClass, isoTopCat, keyword, dateIssued,
                temporalCoverage, dateRange, updateFrequency, spatialCoverage, resourceType,
                formatElement, information, download, idElement, identifier, 
                provider, code, memberOf, status, accrualMethod, dateAccessioned, 
                rights, license, accessRights,
                suppressed, childRecord, fileSize]
    
    return metadata

In [7]:
# iterate each data url to extract metadata
all_metadata = []
count = 0
for url in data_urls:
    count += 1
    print('>>> [{}/{}] harvesting dataset:\n{}'.format(count, len(data_urls), url))
    all_metadata.append(collect_metadata(url))
    # remove datasets without available download files in Shapefile, 
    # ARC/INFO Grid, GeoTIFF, Geodatabase, Geopackage format
    all_metadata = [x for x in all_metadata if x[17]]

>>> [1/584] harvesting dataset:
https://data.humdata.org/dataset/flood-mapping-in-cantua-creek-california-january-2023
>>> [2/584] harvesting dataset:
https://data.humdata.org/dataset/flood-mapping-by-drones-in-merced-california-january-2023
>>> [3/584] harvesting dataset:
https://data.humdata.org/dataset/kontur-population-united-states-of-america
>>> [4/584] harvesting dataset:
https://data.humdata.org/dataset/kontur-boundaries-united-states
>>> [5/584] harvesting dataset:
https://data.humdata.org/dataset/worldpop-population-density-for-united-states-of-america
>>> [6/584] harvesting dataset:
https://data.humdata.org/dataset/worldpop-population-counts-for-united-states-of-america
>>> [7/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_oklahoma_airports
>>> [8/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_oklahoma_sea_ports
>>> [9/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_oklahoma_points_of_interest
>>> [10/584] harv

>>> [82/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_colorado_buildings
>>> [83/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_colorado_points_of_interest
>>> [84/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_wyoming_airports
>>> [85/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_wyoming_points_of_interest
>>> [86/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_wyoming_waterways
>>> [87/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_wyoming_education_facilities
>>> [88/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_wyoming_health_facilities
>>> [89/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_wyoming_financial_services
>>> [90/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_wyoming_roads
>>> [91/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_wyoming_populated_places


>>> [165/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_orgeon_financial_services
>>> [166/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_orgeon_education_facilities
>>> [167/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_orgeon_populated_places
>>> [168/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_orgeon_waterways
>>> [169/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_orgeon_railways
>>> [170/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_orgeon_roads
>>> [171/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_orgeon_buildings
>>> [172/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_idaho_airports
>>> [173/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_idaho_sea_ports
>>> [174/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_idaho_points_of_interest
>>> [175/584] harvestin

>>> [246/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_virginia_roads
>>> [247/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_virginia_buildings
>>> [248/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_virginia_waterways
>>> [249/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_westvirginia_points_of_interest
>>> [250/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_westvirginia_waterways
>>> [251/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_westvirginia_airports
>>> [252/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_westvirginia_sea_ports
>>> [253/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_westvirginia_health_facilities
>>> [254/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_westvirginia_education_facilities
>>> [255/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_we

>>> [326/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_rhodeisland_waterways
>>> [327/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_rhodeisland_points_of_interest
>>> [328/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_rhodeisland_roads
>>> [329/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_rhodeisland_buildings
>>> [330/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_rhodeisland_airports
>>> [331/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_rhodeisland_sea_ports
>>> [332/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_rhodeisland_education_facilities
>>> [333/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_rhodeisland_health_facilities
>>> [334/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_rhodeisland_financial_services
>>> [335/584] harvesting dataset:
https://data.humdata.org/dataset/h

>>> [408/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_georgia_financial_services
>>> [409/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_georgia_education_facilities
>>> [410/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_georgia_railways
>>> [411/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_georgia_populated_places
>>> [412/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_georgia_buildings
>>> [413/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_georgia_roads
>>> [414/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_alabama_points_of_interest
>>> [415/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_alabama_airports
>>> [416/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_alabama_sea_ports
>>> [417/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_alabama_health_facilities
>>

>>> [490/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_wisconsin_populated_places
>>> [491/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_indiana_airports
>>> [492/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_indiana_sea_ports
>>> [493/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_indiana_health_facilities
>>> [494/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_indiana_waterways
>>> [495/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_indiana_education_facilities
>>> [496/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_indiana_points_of_interest
>>> [497/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_indiana_financial_services
>>> [498/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_indiana_railways
>>> [499/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_indiana_popula

>>> [572/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_texas_health_facilities
>>> [573/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_texas_buildings
>>> [574/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_texas_railways
>>> [575/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_texas_points_of_interest
>>> [576/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_texas_waterways
>>> [577/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_texas_airports
>>> [578/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_texas_populated_places
>>> [579/584] harvesting dataset:
https://data.humdata.org/dataset/hotosm_usa_texas_roads
>>> [580/584] harvesting dataset:
https://data.humdata.org/dataset/united-states-high-resolution-population-density-maps-demographic-estimates
>>> [581/584] harvesting dataset:
https://data.humdata.org/dataset/cdc-historical-zika-

### STEP 3: Write a CSV Report

In [8]:
fieldnames = ['Title', 'Alternative Title', 'Description', 'Language', 'Creator', 'Resource Class',
              'ISO Topic Categories', 'Keyword', 'Date Issued', 'Temporal Coverage', 'Date Range', 'Update Frequency', 'Spatial Coverage',
              'Resource Type', 'Format', 'Information', 'Download', 'ID', 'Identifier', 'Provider', 'Code', 'Member Of', 'Status',
              'Accrual Method', 'Date Accessioned', 'Rights', 'License', 'Access Rights', 'Suppressed', 'Child Record', 'File Size']

In [9]:
with open('All_Metadata.csv', 'w') as fw:
    writer = csv.writer(fw)
    writer.writerow(fieldnames)
    writer.writerows(all_metadata)